In [1]:
import tensorflow as tf
import datetime
import os

import numpy as np
import matplotlib.pyplot as plt

2023-05-30 22:33:16.228617: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train, validation = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/train',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224),
    seed=815,
    validation_split=0.1,
    subset='both'
)

test = tf.keras.utils.image_dataset_from_directory(
    '../Dataset/test',
    labels='inferred',
    label_mode='categorical',
    image_size=(224,224)
)

def prepare(dataset):
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.5),
        tf.keras.layers.RandomTranslation(0.33, 0.33),
        tf.keras.layers.RandomZoom(0.33),
    ])

    augmented = dataset
    for _ in range(99):
        augmented = augmented.concatenate(
            dataset.map(
                lambda x, y: (data_augmentation(x, training=True), y),
                num_parallel_calls=tf.data.AUTOTUNE
            )
        )
        
    return augmented.prefetch(buffer_size=tf.data.AUTOTUNE)


train = prepare(train)

Found 22573 files belonging to 25 classes.
Using 20316 files for training.
Using 2257 files for validation.
Found 2500 files belonging to 25 classes.


2023-05-30 22:33:17.972649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 22:33:17.977095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 22:33:17.977259: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
if not os.path.isdir('models'):
    os.mkdir('models')

In [4]:
def generate_model(conv_layers, dense_layers):    
    img_input = tf.keras.layers.Input(shape=(224, 224, 3))
    x = tf.keras.layers.BatchNormalization()(img_input)
    
    for filters, kernel_size, depth, pool in conv_layers:
        shortcut = tf.keras.layers.Conv2D(filters, 1, activation='relu')(x)
        shortcut = tf.keras.layers.BatchNormalization()(shortcut)
        
        for i in range(depth):
            x = tf.keras.layers.Conv2D(                
                filters,
                kernel_size,
                activation='relu',
                padding='same'
            )(x)
            x = tf.keras.layers.BatchNormalization()(x)
                
        x = tf.keras.layers.Add()([shortcut,x])
        x = tf.keras.layers.Activation('relu')(x)
            
        if pool > 1:
            x = tf.keras.layers.MaxPool2D(
                pool_size=pool
            )(x)
            
        x = tf.keras.layers.BatchNormalization()(x)
        
    x = tf.keras.layers.Flatten()(x)
    
    for units in dense_layers:
        x = tf.keras.layers.Dense(
            units,
            activation='relu'
        )(x)
    
    x = tf.keras.layers.Dense(
        25,
        activation='softmax'
    )(x)
    
    
    model = tf.keras.Model(img_input, x)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.000001),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
def run(id):
    model = generate_model(
        [
            # filters, kernel_size, depth, pool
            (32, 7, 2, 2),
            (64, 5, 3, 2),
            (128, 3, 4, 2),
        ], 
        [
            # units
            128,
            192,
        ])
    model.summary()
    
    model.load_weights('checkpoints/20230530-005310')
        
    model.fit(
        train,
        validation_data=validation,
        epochs=5, 
        callbacks=[
            tf.keras.callbacks.TensorBoard(
                log_dir=f'logs/fit/{id}'
            ),
            tf.keras.callbacks.ModelCheckpoint(
                filepath=f'checkpoints/{id}',
                monitor='val_accuracy',
            )
        ]
    )
        
    model.evaluate(
        test, 
        callbacks=[
            tf.keras.callbacks.TensorBoard(
                log_dir=f'logs/evaluate/{id}'
            )
        ]
    )
    

now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
run(f'{now}')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 3)  12         ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 32  4736        ['batch_normalization[0][0]']    
                                )                                                             

                                                                                                  
 conv2d_10 (Conv2D)             (None, 56, 56, 128)  147584      ['batch_normalization_12[0][0]'] 
                                                                                                  
 batch_normalization_13 (BatchN  (None, 56, 56, 128)  512        ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_7 (Conv2D)              (None, 56, 56, 128)  8320        ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 56, 56, 128)  147584      ['batch_normalization_13[0][0]'] 
                                                                                                  
 batch_nor

2023-05-30 22:33:31.724955: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open checkpoints/20230530-005310: FAILED_PRECONDITION: checkpoints/20230530-005310; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?
2023-05-30 22:33:32.351471: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1624' with dtype resource
	 [[{{node Placeholder/_1624}}]]
2023-05-30 22:33:32.366050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_922' with dtype resource
	 [[{{node Placeholder/_922}}]]


Epoch 1/5


2023-05-30 22:33:44.955610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-30 22:33:48.141796: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f89af3d9990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-30 22:33:48.141821: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-05-30 22:33:48.144829: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-30 22:33:48.237189: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 8359/63500 [==>...........................] - ETA: 3:46:10 - loss: 0.4020 - accuracy: 0.8876